# Read Source table

In [0]:
%sql
SELECT * FROM workspace.bronze.crm_cust_info LIMIT 5

# Drop the target table if already exists

In [0]:
%sql
DROP TABLE IF EXISTS workspace.silver.crm_customers

# Create the target table

In [0]:
%sql

CREATE OR REPLACE TABLE workspace.silver.crm_customers
(
  customer_id INT,
  customer_number STRING,
  customer_name STRING,
  marital_status STRING,
  gender STRING,
  create_date DATE
) USING DELTA 
  

# Do the Transformations in the original table & insert to the target table

### Trimming, Normalization, Removing records where customer_id is Null

In [0]:
%sql
INSERT OVERWRITE workspace.silver.crm_customers 
SELECT 
  cst_id AS customer_id,
  cst_key AS customer_number,
  CASE 
    WHEN cst_firstname IS NULL AND cst_lastname IS NULL THEN 'n/a'
    ELSE CONCAT(COALESCE(TRIM(cst_firstname), 'n/a'), ' ', COALESCE(TRIM(cst_lastname), 'n/a')) 
  END AS customer_name,
  CASE 
    WHEN TRIM(cst_marital_status) = 'M' THEN 'Married'
    WHEN TRIM(cst_marital_status) = 'S' THEN 'Single'
    ELSE 'n/a'
  END AS  marital_status,
  CASE 
    WHEN TRIM(cst_gndr) = 'M' THEN 'Male'
    WHEN TRIM(cst_gndr) = 'F' THEN 'Female'
    ELSE 'n/a'
  END AS gender,
  cst_create_date AS create_date
FROM (
SELECT *,
 row_number()over(partition by cst_id order by cst_create_date) RN from workspace.bronze.crm_cust_info
 WHERE CST_ID IS NOT NULL)
 WHERE RN = 1;

# Sanity Checks of Target Table

In [0]:
%sql
SELECT * FROM workspace.silver.crm_customers LIMIT 5

# View Target Table changes

In [0]:
%sql
DESCRIBE HISTORY workspace.silver.crm_customers 